In [1]:
!python -V

Python 3.12.2


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:

import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/08/02 19:41:26 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/08/02 19:41:26 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025/08/02 19:41:26 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/henri/Desktop/Ideias/MLOps/zoomcamp/mlops-zoomcamp/mlruns/1', creation_time=1754160086452, experiment_id='1', last_update_time=1754160086452, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [7]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715210601492

In [14]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():
    mlflow.set_tag("developer", "henrique")
    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")
    mlflow.log_param("model", "Lasso")
    mlflow.log_param("random_state", 42)

    alpha = 0.1
    mlflow.log_param("alpha",alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    print(f"rmse: {rmse}")



rmse: 12.143233939842847


In [20]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [18]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params = params, 
            dtrain = train,
            num_boost_round = 100,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
        return {"loss": rmse, "status": STATUS_OK}



In [ ]:
search_space = {
    "max_depth":scope.int(hp.quniform("max_depth", 4, 10, 1)),
    "learning_rate":hp.loguniform("learning_rate", -3, 0),
    "reg_alpha":hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda":hp.loguniform("reg_lambda", -6, 1),
    "min_child_weight":hp.loguniform("iiiii", -1, 3),
    "objective":"reg:linear",
    "seed":42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
)

[0]	validation-rmse:9.59063                           
[1]	validation-rmse:8.19710                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [20:46:29] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[2]	validation-rmse:7.49424                           
[3]	validation-rmse:7.15202                           
[4]	validation-rmse:6.98138                           
[5]	validation-rmse:6.89264                           
[6]	validation-rmse:6.84566                           
[7]	validation-rmse:6.81882                           
[8]	validation-rmse:6.80234                           
[9]	validation-rmse:6.79216                           
[10]	validation-rmse:6.78508                          
[11]	validation-rmse:6.78286                          
[12]	validation-rmse:6.78012                          
[13]	validation-rmse:6.77265                          
[14]	validation-rmse:6.77030                          
[15]	validation-rmse:6.76713                          
[16]	validation-rmse:6.76517                          
[17]	validation-rmse:6.76295                          
[18]	validation-rmse:6.76111                          
[19]	validation-rmse:6.75936                          
[20]	valid

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [20:46:35] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[3]	validation-rmse:7.22843                                                    
[4]	validation-rmse:7.02828                                                    
[5]	validation-rmse:6.92316                                                    
[6]	validation-rmse:6.86356                                                    
[7]	validation-rmse:6.82846                                                    
[8]	validation-rmse:6.80602                                                    
[9]	validation-rmse:6.78715                                                    
[10]	validation-rmse:6.77881                                                   
[11]	validation-rmse:6.77541                                                   
[12]	validation-rmse:6.77242                                                   
[13]	validation-rmse:6.76531                                                   
[14]	validation-rmse:6.76125                                                   
[15]	validation-rmse:6.75834            

...

[1]	validation-rmse:7.45521                                                     
[2]	validation-rmse:7.01514                                                     
[3]	validation-rmse:6.85461                                                     
[4]	validation-rmse:6.78584                                                     
[5]	validation-rmse:6.75406                                                     
[6]	validation-rmse:6.74057                                                     
[7]	validation-rmse:6.73331                                                     
[8]	validation-rmse:6.72719                                                     
[9]	validation-rmse:6.72101                                                     
[10]	validation-rmse:6.71630                                                    
[11]	validation-rmse:6.71454                                                    
[12]	validation-rmse:6.70948                                                    
[13]	validation-rmse:6.70624

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [20:54:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:9.41913                                                     
[1]	validation-rmse:8.00749                                                     
[2]	validation-rmse:7.33944                                                     
[3]	validation-rmse:7.02217                                                     
[4]	validation-rmse:6.86826                                                     
[5]	validation-rmse:6.79196                                                     
[6]	validation-rmse:6.74852                                                     
[7]	validation-rmse:6.72502                                                     
[8]	validation-rmse:6.71047                                                     
[9]	validation-rmse:6.70144                                                     
[10]	validation-rmse:6.69669                                                    
[11]	validation-rmse:6.69260                                                    
[12]	validation-rmse:6.69071

c:\Users\henri\Desktop\Ideias\MLOps\zoomcamp\mlops-zoomcamp\venv\Lib\site-packages\xgboost\callback.py:386: UserWarning: [20:54:25] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\objective\regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()



[0]	validation-rmse:7.86479                                                     
[1]	validation-rmse:6.97886                                                     
[2]	validation-rmse:6.79563                                                     
[3]	validation-rmse:6.74243                                                     
[4]	validation-rmse:6.72449                                                     
[5]	validation-rmse:6.71782                                                     
[6]	validation-rmse:6.71048                                                     
[7]	validation-rmse:6.70338                                                     
[8]	validation-rmse:6.69790                                                     
[9]	validation-rmse:6.69295                                                     
[10]	validation-rmse:6.69014                                                    
[11]	validation-rmse:6.68748                                                    
[12]	validation-rmse:6.68138